In [1]:
import numpy as np
import pandas as pd
import re

from scipy import stats
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

1.- Data Exploration

In [2]:
data = pd.read_csv('Shark_attacks_clean.csv')
data.head(2)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
0,2016-09-18,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16.0,Minor injury to thigh,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016-09-18,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43.0,Lacerations to lower leg,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5807 entries, 0 to 5806
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    5807 non-null   object 
 1   Type                    5807 non-null   object 
 2   Country                 5807 non-null   object 
 3   Area                    5807 non-null   object 
 4   Location                5693 non-null   object 
 5   Activity_during_attack  5448 non-null   object 
 6   Victim_Name             5807 non-null   object 
 7   Gender                  5269 non-null   object 
 8   Age                     5126 non-null   float64
 9   Injury                  5807 non-null   object 
 10  Fatal_attack            5807 non-null   bool   
 11  Shark_species           5807 non-null   object 
 12  Source_information      5807 non-null   object 
 13  Documentation_pdf       5807 non-null   object 
 14  Documentation_href      5807 non-null   

1) There are no columns 22 & 23 to be removed

2) Columns to change type:
    0-Date to datestamp
    8-Age to int64

3) Check duplicated rows

In [4]:
(data[[column for column in list(data.columns)]][data[[column for column in list(data.columns)]].duplicated()])

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
5096,1923-01-01,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,False,Unidentified,"Ref in New York Herald Tribune, 8/23/1960; V.M...",1923.00.00.a-NJ fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


4) Only row 5096 appears to be twice in the dataset, if so, we can drop the row by filtering location

In [5]:
data[data.loc[:,'Location'] == 'Ocean City (offshore)']

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
5095,1923-01-01,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,False,Unidentified,"Ref in New York Herald Tribune, 8/23/1960; V.M...",1923.00.00.a-NJ fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5096,1923-01-01,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,False,Unidentified,"Ref in New York Herald Tribune, 8/23/1960; V.M...",1923.00.00.a-NJ fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [6]:
data = data.drop(5095, axis=0)

In [7]:
# Just verifying the row has been succesfully dropped
data[data.loc[:,'Location'] == 'Ocean City (offshore)']

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
5096,1923-01-01,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,False,Unidentified,"Ref in New York Herald Tribune, 8/23/1960; V.M...",1923.00.00.a-NJ fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


5) Now check how many null data is in dataset

In [8]:
# checar valores nulos en columnas
null_cols = data.isnull().sum()
null_cols

Date                        0
Type                        0
Country                     0
Area                        0
Location                  114
Activity_during_attack    359
Victim_Name                 0
Gender                    538
Age                       680
Injury                      0
Fatal_attack                0
Shark_species               0
Source_information          0
Documentation_pdf           0
Documentation_href          0
dtype: int64

In [9]:
# only 4 columns with null data
null_cols[null_cols > 0]

Location                  114
Activity_during_attack    359
Gender                    538
Age                       680
dtype: int64

In [10]:
# turning nulls into percetage 
null_cols = data.isnull().mean()
null_cols = null_cols[null_cols > 0].sort_values() * 100
null_cols

Location                   1.963486
Activity_during_attack     6.183259
Gender                     9.266276
Age                       11.712022
dtype: float64

6) "Due to the relevance of the "Species" column in the data set, we cannot drop this column, even when the percentage of NaN's there is near to 50%."

But 'Species' has no null data, most of them are 'Unidentified'

7) "The column "Time" and "Year" seems to be irrelevant. However we will explore it further"

There are no columns 'Time' or 'Year' but maybe it will be necesary to obtain year after cleaning 'Date' column

8) "The columns "Country", "Area" and "Location" are related, so it might be possible to infer the missing values one from the others"

Lets check it

In [11]:
# Checking nulls in 'Location'
data[(data.Country.isna() == False) & (data.Area.isna()== False) & (data.Location.isna())].head(2)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
54,2016-06-24,Unprovoked,COLOMBIA,Isla Provedencia,NaN,Diving,Arturo Velez,M,59.0,Severe bite to right hand,False,"Caribbean reef shark, 4.5'",Dr. A. Velez,2016.06.24-Velez.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
86,2016-04-08,Invalid,CAPE VERDE,Boa Vista Island,NaN,Swimming,a British citizen,M,60.0,"""Serious""",False,Shark involvement not confirmed,L.O.Guttke,2016.04.08-CapeVerde.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [12]:
# list of the rows indexes for null values in 'Location' column
null_rows = list(data[(data.Country.isna() == False) & (data.Area.isna()== False) & (data.Location.isna())].index)
# list of the areas for null values in 'Location' column
areas = list(data[((data.Country.isna() == False) & (data.Area.isna() == False) & (data.Location.isna()))]["Area"].loc[null_rows])

# 'for loop' to check if we can fill null values in 'Location' by areas
b = []
for i in range(0,len(areas)):
    a = data[data.loc[:,'Area'] == areas[i]]
    indexes = len(a)
    # this 'if' filters 'a' when area is in more than 1 index in original 'data'
    if indexes > 1:
        b.append(a)
# dataframe with repeated areas
b = pd.concat(b, axis=0)

In [13]:
b.head(2)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
350,2014-06-01,Unprovoked,USA,Palmyra Atoll,NaN,Tagging,female,F,37.0,Laceration to left hand,False,Blacktip Reef shark,U.S. Coast Guard,2014.06.01.a-PalmyraAtoll.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
800,2010-11-19,Provoked,USA,Palmyra Atoll,NaN,Snorkeling,Kydd Pollock,M,33.0,Head bitten by netted shark PROVOKED INCIDENT,False,Unidentified,"Sunday News, 12/12/2010",2010.11.19-Pollock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


9) Unfortunatelly we cannot infer locations by repeating areas. 
We will leave these rows by now and see if we can drop this null values later

10) Capitalizing 'Country' column

In [14]:
data["Country"] = data['Country'].str.capitalize() 

In [15]:
data.head(2)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
0,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16.0,Minor injury to thigh,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43.0,Lacerations to lower leg,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


11) 'Cleaning and parsing the column "Date"'

'Date' column appears to be ok overall. No cleaning needed

12) 'Droping the column "Case Number", "Year" and "original order"'

"Case Number", "Year" and "original order" columns are not in dataset.

13) 'Filling values for "Name" and "Investigator or Source"'

We check if 'Name' can be infered by other columns

In [16]:
data['Victim_Name'].unique()

array(['male', 'Chucky Luciano', 'Rory Angiolella', ...,
       'Samuel Jennings, a deserter from the British frigate Milford',
       'seaman', 'seaman from the York'], dtype=object)

14) We 'could' fill the names by searching patterns in the 'Documentation-href' column, but this would take a lot of time due to the many different text entries in 'Victim_Name' column.
Therefore, we will leave 'Victim_Name' as it is.
'Source' column needs no filling either.

15) Tiding column "Activity" by extracting verbs in gerund.

In [17]:
# obtaining list with rows in which no age nor activity have data,
# We will delete this rows from the dataset since they will not work with the predictor function
data_temp = list(data[(data['Activity_during_attack'].isnull()== True) & (data['Age'].isnull()== True)].index)
data = data.drop(data_temp, axis=0)

In [18]:
# this list will contain all activities, even those with long texts
activity_list = list(data['Activity_during_attack'].unique())
# regex pattern for gerund verbs
pattern = r'\b(\w+ing)\b'
gerund_verbs_list = []

# this for loop will deliver only the -ing gerung verbs
for i in range(0,len(activity_list)):
    x = re.findall(r'\b(\w+ing)\b', str(activity_list[i]))
    gerund_verbs_list.append(x)
    
# this for loop deliveres the last list but clean
gerund_verbs_list_clean = []
for i in gerund_verbs_list:
    for j in i:
        gerund_verbs_list_clean.append(j)

In [19]:
# we will use the last 'gerund_verbs_list_clean' list to filter 
# and eliminate activities with no useful data for the preductor function
data = data.loc[data['Activity_during_attack'].isin(gerund_verbs_list_clean)]
data.head(2)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
0,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16.0,Minor injury to thigh,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43.0,Lacerations to lower leg,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [20]:
# checking nulls percentage drop
null_cols = data.isnull().mean()
null_cols = null_cols[null_cols > 0].sort_values() * 100
null_cols

Location    1.411488
Age         1.901588
Gender      6.567340
dtype: float64

16) Create a Predictor Function and with it, filling NaN values in Columns "Age" and "Activity".
For this function we need a frequency list, a dataframe filter based in the frequency list

In [21]:
# replacing nulls in 'Age' to zeros
data['Age'] = data['Age'].fillna(0)
# Changing 'Age' from float64 to int64
data['Age'] = data['Age'].astype(np.int64)

In [22]:
# frequency list, first for 'Age'
age_frequency = list(data['Age'].value_counts().index)
# dataframe filter, this will create a list with count - age
data_filter = data[data['Age'].isin(age_frequency)]['Age'].value_counts()
most_frequent = list(data_filter.index)[0]

In [23]:
# in the dataset, 18 years is the most frequent age and fishing the most common activity
x = (most_frequent, data[(data['Age'].isin(age_frequency)) 
                         & (data['Age'] == most_frequent)]['Activity_during_attack'].value_counts())
x

(18,
 Fishing         476
 Surfing          43
 Swimming         31
 Floating         16
 Treading          9
 Boarding          7
 Diving            7
 Spearfishing      6
 Wading            5
 Bathing           4
 Standing          4
 Filming           2
 Snorkeling        2
 Walking           1
 Playing           1
 Scurfing          1
 Splashing         1
 Paddling          1
 Jumping           1
 Holding           1
 Name: Activity_during_attack, dtype: int64)

In [24]:
# this function will search through all the most common ages by activity
def predictor(column1, column2, dataframe):
    age_frequency = list(dataframe[column1].value_counts().index)
    data_filter = dataframe[dataframe[column1].isin(age_frequency)][column2].value_counts()
    a = {}
    for i in list(data_filter.index):
        a[i] = dataframe[dataframe[column1].isin(age_frequency)][column1].value_counts().index[0]
    return a

In [25]:
predictor('Age', 'Activity_during_attack', data)

{'Surfing': 18,
 'Swimming': 18,
 'Fishing': 18,
 'Diving': 18,
 'Spearfishing': 18,
 'Bathing': 18,
 'Wading': 18,
 'Boarding': 18,
 'Standing': 18,
 'Snorkeling': 18,
 'Floating': 18,
 'Treading': 18,
 'Kayaking': 18,
 'Skiing': 18,
 'Attempting': 18,
 'Walking': 18,
 'Playing': 18,
 'Sitting': 18,
 'Windsurfing': 18,
 'Rowing': 18,
 'Paddling': 18,
 'Sinking': 18,
 'Canoeing': 18,
 'Sailing': 18,
 'Freediving': 18,
 'Collecting': 18,
 'Skindiving': 18,
 'Washing': 18,
 'Netting': 18,
 'Splashing': 18,
 'During': 18,
 'Jumping': 18,
 'Boating': 18,
 'Feeding': 18,
 'Hunting': 18,
 'Lying': 18,
 'Clinging': 18,
 'Removing': 18,
 'Filming': 18,
 'Watching': 18,
 'Competing': 18,
 'Crossing': 18,
 'Lifesaving': 18,
 'Tagging': 18,
 'Dangling': 18,
 'Paddleskiing': 18,
 'Clamming': 18,
 'Paddleboarding': 18,
 'Returning': 18,
 'Being': 18,
 'Riding': 18,
 'Crabbing': 18,
 'Catching': 18,
 'Pulling': 18,
 'Spearing': 18,
 'Cleaning': 18,
 'Escaping': 18,
 'Dragging': 18,
 'Sculling': 18,


17) Based on the results of the last output, we can substitute all the '0' in 'Age' column for '18'

In [26]:
data['Age'] = data['Age'].replace(0,18)

In [31]:
# verifying there are no more '0' in 'Age'
data[data.loc[:,'Age'] == 0]

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href


In [32]:
data['Age'].unique()

array([16, 43, 36, 17, 18, 65, 51, 50, 12,  9, 22, 37, 25, 20, 49, 15, 21,
       40, 72, 27, 29, 28, 31, 11, 30, 24, 10, 59, 42, 34, 35, 26, 13, 19,
        6, 60, 64, 23, 52, 57, 48, 39, 69, 46, 41, 45, 38, 71, 32, 55, 54,
       44, 14,  7, 62, 68, 47, 63, 70, 58,  1, 53, 33,  8, 61, 66, 77, 74,
        3, 56,  5, 86, 84, 75, 73, 87, 67, 81, 78], dtype=int64)

In [33]:
# checking nulls percentage drop
null_cols = data.isnull().mean()
null_cols = null_cols[null_cols > 0].sort_values() * 100
null_cols

Location    1.411488
Gender      6.567340
dtype: float64

18) Location cannot be inferred with other columns, we could drop these values, but since is more than 1% we will let them and fill the nulls with 'Unknown
Gender could be inferred with the names in 'Victim_Name', although we will fill nulls with 'Unknown' since the nulls are considerably more than 5% of the toal entries

In [36]:
data['Location'] = data['Location'].fillna('Unknown')
data['Gender'] = data['Gender'].fillna('Unknown')

In [38]:
# checking nulls percentage drop
null_cols = data.isnull().mean()
null_cols = null_cols[null_cols > 0].sort_values() * 100
null_cols

Series([], dtype: float64)

In [48]:
# reseting index
data.reset_index(drop=True)

,Date,Type,Country,Area,Location,Activity_during_attack,Victim_Name,Gender,Age,Injury,Fatal_attack,Shark_species,Source_information,Documentation_pdf,Documentation_href
0,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016-09-18,Unprovoked,Usa,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,False,Unidentified,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016-09-17,Unprovoked,Australia,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,17,Struck by fin on chest & leg,False,Unidentified,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016-09-16,Unprovoked,Australia,Victoria,Bells Beach,Surfing,male,M,17,No injury: Knocked off board by shark,False,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5096,1738-04-06,Unprovoked,Italy,Sicily,Strait of Messina,Swimming,male,M,16,FATAL,True,Unidentified,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5097,1721-06-01,Unprovoked,Italy,Sardinia,"Ponte della Maddelena,",Swimming,male,M,16,"FATAL, partial remains recovered from sharks...",True,"White shark, 1600-lb female",F. Ricciardi; A. De Maddalena.,1721.06.00-Maddalena.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5098,1703-03-26,Unprovoked,Barbados,Southwest coast,Carlisle Bay,Swimming,"Samuel Jennings, a deserter from the British f...",M,19,"Hand and foot severely bitten, surgically ampu...",False,Unidentified,"W.R.Cutter, Vol.1, p.252",1703.03.26-Jennings.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5099,1700-01-01,Unprovoked,France,Côte d'Azur,Antibes,Bathing,seaman,M,17,Leg severed,False,White shark,"A. De Maddalena, citing Cazeils (1998)",1700.00.00.b-Antibes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5101 entries, 0 to 5806
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    5101 non-null   object
 1   Type                    5101 non-null   object
 2   Country                 5101 non-null   object
 3   Area                    5101 non-null   object
 4   Location                5101 non-null   object
 5   Activity_during_attack  5101 non-null   object
 6   Victim_Name             5101 non-null   object
 7   Gender                  5101 non-null   object
 8   Age                     5101 non-null   int64 
 9   Injury                  5101 non-null   object
 10  Fatal_attack            5101 non-null   bool  
 11  Shark_species           5101 non-null   object
 12  Source_information      5101 non-null   object
 13  Documentation_pdf       5101 non-null   object
 14  Documentation_href      5101 non-null   object
dtypes: b

In [51]:
# saving data to csv file
data.to_csv("./Shark_attacks_cleaned.csv", index= False)